In [350]:
import numpy as np
import pandas as pd

redfin_df = pd.read_csv("redfin_cleaned.csv")

redfin_df["source"] = "redfin"

redfin_df = redfin_df.iloc[:,[27,3,4,5,6,8,9,20,7]]

redfin_df_mapping = {redfin_df.columns[0]: "source",
                     redfin_df.columns[1]: "address", 
                     redfin_df.columns[2]: "city", 
                     redfin_df.columns[3]: "state_or_province", 
                     redfin_df.columns[4]: "zipcode", 
                     redfin_df.columns[5]: "beds", 
                     redfin_df.columns[6]: "baths", 
                     redfin_df.columns[7]: "url", 
                     redfin_df.columns[8]: "price"}
                     
redfin_df=redfin_df.rename(columns=redfin_df_mapping)

In [323]:
zillow_df = pd.read_csv("zillow_cleaned.csv")

zillow_df["source"] = "zillow"

zillow_df = zillow_df.iloc[:,[11,2,0,1,4,6,5,9,8]]

zillow_df_mapping = {zillow_df.columns[0]: "source",
                     zillow_df.columns[1]: "address", 
                     zillow_df.columns[2]: "city", 
                     zillow_df.columns[3]: "state_or_province", 
                     zillow_df.columns[4]: "zipcode", 
                     zillow_df.columns[5]: "beds", 
                     zillow_df.columns[6]: "baths", 
                     zillow_df.columns[7]: "url", 
                     zillow_df.columns[8]: "price"}

zillow_df=zillow_df.rename(columns=zillow_df_mapping)
zillow_df
merged_df = pd.concat([redfin_df, zillow_df], axis=0)

In [324]:
# Get in range
merged_df = merged_df.dropna(subset=['beds']) # gets rid of nas
merged_df = merged_df.loc[(merged_df['beds']>0)] # get rid of places with no beds listed
merged_df = merged_df.loc[(merged_df['price']>0)] # get rid of places that have no price listed
merged_df = merged_df.groupby(['beds']).filter(lambda x: len(x) >= 30) # get rid of bed groups with less than 30 listings
merged_df

,source,address,city,state_or_province,zipcode,beds,baths,url,price
0,redfin,1062 Lindendale Dr,Mt. Lebanon,PA,15243,4.0,2.5,http://www.redfin.com/PA/Pittsburgh/1062-Linde...,349000
1,redfin,401 Lorenz Ave,Elliott,PA,15220,3.0,2.0,http://www.redfin.com/PA/Pittsburgh/401-Lorenz...,120000
2,redfin,307 S Dithridge St #505,Oakland,PA,15213,2.0,2.0,http://www.redfin.com/PA/Pittsburgh/307-S-Dith...,319900
3,redfin,1702 S Hawthorn Ct,Franklin Park,PA,15237,4.0,3.5,http://www.redfin.com/PA/Pittsburgh/1702-S-Haw...,620000
4,redfin,5226 5th Ave #8,Shadyside,PA,15232,3.0,2.0,http://www.redfin.com/PA/Pittsburgh/5226-Fifth...,355000
...,...,...,...,...,...,...,...,...,...
1535,zillow,5583 Butler St,Pittsburgh,PA,15201,3.0,4.0,https://www.zillow.com/homedetails/5583-Butler...,699900
1536,zillow,1912 Funston St,Pittsburgh,PA,15235,3.0,2.0,https://www.zillow.com/homedetails/1912-Funsto...,65000
1538,zillow,112 McNary Way,Pittsburgh,PA,15212,2.0,1.0,https://www.zillow.com/homedetails/112-McNary-...,99000
1539,zillow,420 S Aiken Ave APT B6,Pittsburgh,PA,15232,2.0,1.0,https://www.zillow.com/homedetails/420-S-Aiken...,199900


In [325]:
merged_df['price_adjusted'] = merged_df.groupby('beds').transform(lambda x: (x - x.mean()) / x.std())['price']
merged_df

,source,address,city,state_or_province,zipcode,beds,baths,url,price,price_adjusted
0,redfin,1062 Lindendale Dr,Mt. Lebanon,PA,15243,4.0,2.5,http://www.redfin.com/PA/Pittsburgh/1062-Linde...,349000,-0.115455
1,redfin,401 Lorenz Ave,Elliott,PA,15220,3.0,2.0,http://www.redfin.com/PA/Pittsburgh/401-Lorenz...,120000,-0.526262
2,redfin,307 S Dithridge St #505,Oakland,PA,15213,2.0,2.0,http://www.redfin.com/PA/Pittsburgh/307-S-Dith...,319900,0.378613
3,redfin,1702 S Hawthorn Ct,Franklin Park,PA,15237,4.0,3.5,http://www.redfin.com/PA/Pittsburgh/1702-S-Haw...,620000,0.559213
4,redfin,5226 5th Ave #8,Shadyside,PA,15232,3.0,2.0,http://www.redfin.com/PA/Pittsburgh/5226-Fifth...,355000,0.194820
...,...,...,...,...,...,...,...,...,...,...
1535,zillow,5583 Butler St,Pittsburgh,PA,15201,3.0,4.0,https://www.zillow.com/homedetails/5583-Butler...,699900,1.253122
1536,zillow,1912 Funston St,Pittsburgh,PA,15235,3.0,2.0,https://www.zillow.com/homedetails/1912-Funsto...,65000,-0.695025
1538,zillow,112 McNary Way,Pittsburgh,PA,15212,2.0,1.0,https://www.zillow.com/homedetails/112-McNary-...,99000,-0.697110
1539,zillow,420 S Aiken Ave APT B6,Pittsburgh,PA,15232,2.0,1.0,https://www.zillow.com/homedetails/420-S-Aiken...,199900,-0.205755


In [326]:
# Summary stats to see how many in each group
merged_df.groupby(by="beds").mean()
num_items_in_groups = merged_df.groupby(['beds']).size()
num_items_in_groups


beds
1.0     115
2.0     581
3.0    1215
4.0     534
5.0     158
6.0      65
dtype: int64

In [349]:
A = [int(x) for x in list(merged_df.groupby(['beds']).groups.keys())]
A


[1, 2, 3, 4, 5, 6]

In [327]:
# Filtering part
groups = merged_df.groupby("beds")['price_adjusted'] 

# Get below 1 std
m = merged_df["price_adjusted"].lt(-0.8) 

new_df = merged_df.loc[m]
# new_df
new_df



,source,address,city,state_or_province,zipcode,beds,baths,url,price,price_adjusted
12,redfin,611 Calais Dr Unit 4104-A,McCandless,PA,15237,1.0,1.0,http://www.redfin.com/PA/Pittsburgh/611-Calais...,113000,-1.025564
42,redfin,42 Norton St,Mt Washington,PA,15211,2.0,1.5,http://www.redfin.com/PA/Pittsburgh/42-Norton-...,31800,-1.024355
51,redfin,1128 South Side,Spring Hill,PA,15212,5.0,1.0,http://www.redfin.com/PA/Pittsburgh/1128-S-Sid...,95000,-1.168656
61,redfin,2908 Glasgow St,Sheraden,PA,15204,1.0,1.0,http://www.redfin.com/PA/Pittsburgh/2908-Glasg...,100000,-1.112247
70,redfin,1987 Centurion Dr #609,Forest Hills Boro,PA,15221,1.0,1.0,http://www.redfin.com/PA/Pittsburgh/1987-Centu...,62900,-1.359628
...,...,...,...,...,...,...,...,...,...,...
1460,zillow,4601 5th Ave APT 328,Pittsburgh,PA,15213,1.0,2.0,https://www.zillow.com/homedetails/4601-5th-Av...,110000,-1.045568
1480,zillow,3333 Milwaukee St,Pittsburgh,PA,15219,5.0,3.0,https://www.zillow.com/homedetails/3333-Milwau...,150000,-1.009789
1486,zillow,1702 High St,Pittsburgh,PA,15212,4.0,1.0,https://www.zillow.com/homedetails/1702-High-S...,39999,-0.884729
1487,zillow,586 Twin Oak Dr,Pittsburgh,PA,15235,1.0,1.0,https://www.zillow.com/homedetails/586-Twin-Oa...,100000,-1.112247


In [341]:
test_df = merged_df[merged_df['beds']==3]
cols = ['beds'] # one or more

Q1 = test_df.price_adjusted.quantile(0.05)

Q1


-0.6646479576691373

In [328]:
new_df.groupby('beds').size()/num_items_in_groups

beds
1.0    0.252174
2.0    0.110155
3.0    0.010700
4.0    0.037453
5.0    0.227848
6.0    0.030769
dtype: float64

In [320]:
new_df.groupby('beds').mean()

,zipcode,baths,price,price_adjusted
beds,,,,
1.0,15219.758621,1.224138,113203.448276,-1.024207
2.0,15214.218750,1.064516,51107.812500,-0.930332
3.0,15212.076923,1.000000,20146.153846,-0.832656
4.0,15216.450000,1.550000,47272.400000,-0.866621
5.0,15213.083333,1.902778,154486.111111,-0.996831
6.0,15215.500000,2.500000,49999.500000,-0.835732
